# Model 1

## haarcascade_frontalface_default.xml

In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
from IPython.display import display

cap=cv2.VideoCapture(0)
face=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
skip=0
face_data=[]
dataset_path='./data/'
file_roll_person=input("enter the roll number:")
stud_phone = input("enter the name:")
df = pd.read_csv('students.csv')
data = {
    "Phone Number" : [str(stud_phone)],
    "Roll Number" :[ str(file_roll_person)]
}
add_df = pd.DataFrame(data)
new_df  = df.append(add_df)
new_df.to_csv('students.csv',index=False)
file_name =  str(file_roll_person)

while True:
	ret,frame=cap.read()
	gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

	if ret==False:
		continue

	faces=face.detectMultiScale(frame,1.3,5)
	faces=sorted(faces,key=lambda f:f[2]*f[3])

	for (x,y,w,h) in faces[-1:]:
		cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)

		offset=10
		face_section=frame[y-offset:y+h+offset,x-offset:x+w+offset]
		face_section=cv2.resize(face_section,(100,100))

		skip+=1
		if skip%10==0:
			face_data.append(face_section)
			print(face_data)

	cv2.imshow("frame",frame)
	#cv2.imshow("face_section",face_section)

	key=cv2.waitKey(30) & 0xFF

	if key==ord('q'):
		break	

face_data=np.asarray(face_data)
face_data=face_data.reshape((face_data.shape[0],-1))

display(df.head())

np.save(dataset_path+file_name+".npy",face_data)
cap.release()
cv2.destroyAllWindows()

In [3]:
import cv2
import os
import numpy as np
import pandas as pd
from IPython.display import display
def knn(X,Y,k=5):
    val=[]
    m=X.shape[0]
    for i in range(m):
        ix=X[i,:-1]
        iy=X[i,-1]

        d=dist(Y,ix)
        val.append((d,iy))
    vals=sorted(val,key=lambda x:x[0])[:k]
    vals=np.array(vals)[:,-1]
    
    new_val=np.unique(vals,return_counts=True)
    #print(new_val)
    index=np.argmax(new_val[1])
    pred=new_val[0][index]
    print(pred)
    return pred
def dist(x1,x2):
    return np.sqrt(sum(((x1-x2)**2)))
def mark_attendance(ids):
    print(ids)
    df = pd.DataFrame({
        'Roll Number' : ids
    })
    df.to_csv('attendance.csv')
    
    new_df = pd.read_csv('attendace.csv')
    columns_list = np.array(new_df.columns)
    drop_col = []
    for col in columns_list:
        if "Unnamed:" in  col:
            drop_col.append(col)
    new_df.drop(drop_col,axis = 1,inplace=True)
    new_df.fillna(0,inplace=True)
    new_df.to_csv('attendace.csv',index=False)
    new_df = pd.read_csv('attendace.csv')
    print(new_df.head())
cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

skip=0
face_data=[]
dataset_path='./data/'
label=[]
class_id=0
uniq_student_ids = []
names={}
students_ids = [ ]
stud_df = pd.read_csv('students.csv')
current_students = [ ]
student_id = ' '
for i in range(stud_df.shape[0]):
    student_id  = str(stud_df['Roll Number'].values[i])
    current_students.append(student_id)
    
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):

        names[class_id]=fx[:-4]

        print("loaded "+fx)
        data_item=np.load(dataset_path+fx)
        face_data.append(data_item)
        #Create labels for class
        target=class_id*np.ones((data_item.shape[0],))
        class_id+=1
        label.append(target)


face_dataset=np.concatenate(face_data,axis=0)
labels_dataset=np.concatenate(label,axis=0).reshape((-1,1))

trainset=np.concatenate((face_dataset,labels_dataset),axis=1)

while True:
    ret,frame=cap.read()

    if ret==False:
        continue

    faces=face_cascade.detectMultiScale(frame,1.3,5)

    for face in faces:
        x,y,w,h=face

        offset=10

        face_section=frame[y-offset:y+h+offset,x-offset:x+offset+w]
        face_section=cv2.resize(face_section,(100,100))
        out=knn(trainset,face_section.flatten())
        pred=names[int(out)]
        students_ids.append(pred)
        cv2.putText(frame,pred,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,1),1,cv2.LINE_AA)

        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)

    cv2.imshow("frame",frame)
    key=cv2.waitKey(1) & 0xFF
    if key==ord('q'):
        break
for ids in students_ids:
    if ids not in uniq_student_ids:
        uniq_student_ids.append(ids)
mark_attendance(uniq_student_ids)
cap.release()
cv2.destroyAllWindows()

loaded 6.npy
loaded 8.npy
1.0
['8']
1.0
['8', '8']
1.0
['8', '8', '8']
1.0
['8', '8', '8', '8']
1.0
['8', '8', '8', '8', '8']
1.0
['8', '8', '8', '8', '8', '8']
1.0
['8', '8', '8', '8', '8', '8', '8']
1.0
['8', '8', '8', '8', '8', '8', '8', '8']
0.0
['8', '8', '8', '8', '8', '8', '8', '8', '6']
1.0
['8', '8', '8', '8', '8', '8', '8', '8', '6', '8']
1.0
['8', '8', '8', '8', '8', '8', '8', '8', '6', '8', '8']
0.0
['8', '8', '8', '8', '8', '8', '8', '8', '6', '8', '8', '6']
1.0
['8', '8', '8', '8', '8', '8', '8', '8', '6', '8', '8', '6', '8']
1.0
['8', '8', '8', '8', '8', '8', '8', '8', '6', '8', '8', '6', '8', '8']
0.0
['8', '8', '8', '8', '8', '8', '8', '8', '6', '8', '8', '6', '8', '8', '6']
0.0
['8', '8', '8', '8', '8', '8', '8', '8', '6', '8', '8', '6', '8', '8', '6', '6']
0.0
['8', '8', '8', '8', '8', '8', '8', '8', '6', '8', '8', '6', '8', '8', '6', '6', '6']
0.0
['8', '8', '8', '8', '8', '8', '8', '8', '6', '8', '8', '6', '8', '8', '6', '6', '6', '6']
0.0
['8', '8', '8', '8', '8',

# Model 2

## haarcascade_frontalface_alt .xml

In [ ]:
import cv2
import numpy as np
import pandas as pd

cap=cv2.VideoCapture(0)
face=cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
skip=0
face_data=[]
dataset_path='./data/'
file_name_person=input("enter the name:")
file_roll_person=input("enter the roll number:")
df = pd.read_csv('attendace.csv')
print(df)
data = {
    "Names" : [str(file_name_person)],
    "Roll Number" :[ str(file_roll_person)]
}
add_df = pd.DataFrame(data)
new_df  = df.append(add_df)
new_df.to_csv('attendace.csv',index=False)

file_name = str(file_name_person) + str(file_roll_person)

while True:
	ret,frame=cap.read()
	gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

	if ret==False:
		continue

	faces=face.detectMultiScale(frame,1.3,5)
	faces=sorted(faces,key=lambda f:f[2]*f[3])

	for (x,y,w,h) in faces[-1:]:
		cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)

		offset=10
        face_section=frame[y-offset:y+h+offset,x-offset:x+offset+w]
        face_section=cv2.resize(face_section,(100,100))
        print(face_section)
        out=knn(trainset,face_section.flatten())
        pred=names[int(out)]
        if pred in current_students:
            print(pred)
            pred = pred + 'Access Allowed'
        else:
            pred = pred + 'Access Denied'
            
		skip+=1
		if skip%10==0:
			face_data.append(face_section)
			print(face_data)

	cv2.imshow("frame",frame)
	#cv2.imshow("face_section",face_section)

	key=cv2.waitKey(30) & 0xFF

	if key==ord('q'):
		break	

face_data=np.asarray(face_data)
face_data=face_data.reshape((face_data.shape[0],-1))
#print(face_data.shape)

new_df = pd.read_csv('attendace.csv')
columns_list = np.array(new_df.columns)
drop_col = []
for col in columns_list:
    if "Unnamed:" in  col:
        drop_col.append(col)
new_df.drop(drop_col,axis = 1,inplace=True)
new_df.fillna(0,inplace=True)
new_df.to_csv('attendace.csv',index=False)
new_df = pd.read_csv('attendace.csv')

print(new_df.head())

np.save(dataset_path+file_name+".npy",face_data)
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
import pandas as pd
import os
def dist(x1,x2):
    return np.sqrt(sum(((x1-x2)**2)))

new_df = pd.read_csv('attendace.csv')
current_students = [ ]
student_id = ' '
for i in range(new_df.shape[0]):
    student_id  = new_df['Names'].values[i] + str(new_df['Roll Number'].values[i])
    current_students.append(student_id)
print(current_students)
def knn(X,Y,k=5):
    val=[]
    m=X.shape[0]
    for i in range(m):
        ix=X[i,:-1]
        iy=X[i,-1]

        d=dist(Y,ix)
        val.append((d,iy))
    vals=sorted(val,key=lambda x:x[0])[:k]
    vals=np.array(vals)[:,-1]
    
    new_val=np.unique(vals,return_counts=True)
    #print(new_val)
    index=np.argmax(new_val[1])
    pred=new_val[0][index]
    return pred


cap=cv2.VideoCapture(0)

face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

skip=0
face_data=[]
dataset_path='./data/'
label=[]
class_id=0
names={}

for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):

        names[class_id]=fx[:-4]

        print("loaded "+fx)
        data_item=np.load(dataset_path+fx)
        face_data.append(data_item)


        #Create labels for class
        target=class_id*np.ones((data_item.shape[0],))
        class_id+=1
        label.append(target)


face_dataset=np.concatenate(face_data,axis=0)
labels_dataset=np.concatenate(label,axis=0).reshape((-1,1))

trainset=np.concatenate((face_dataset,labels_dataset),axis=1)

while True:
    ret,frame=cap.read()

    if ret==False:
        continue

    faces=face_cascade.detectMultiScale(frame,1.3,5)

    for face in faces:
        x,y,w,h=face

        offset=10

        face_section=frame[y-offset:y+h+offset,x-offset:x+offset+w]
        face_section=cv2.resize(face_section,(100,100))
        print(face_section)
        out=knn(trainset,face_section.flatten())
        pred=names[int(out)]
        if pred in current_students:
            print(pred)
            pred = pred + 'Access Allowed'
        else:
            pred = pred + 'Access Denied'
        cv2.putText(frame,pred,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,1),1,cv2.LINE_AA)

        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)

    cv2.imshow("frame",frame)
    key=cv2.waitKey(1) & 0xFF
    if key==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Model 2

## haarcascade_frontalface_alt2 .xml

In [ ]:
import cv2
import numpy as np
import pandas as pd

cap=cv2.VideoCapture(0)
face=cv2.CascadeClassifier("haarcascade_frontalface_alt2.xml")
skip=0
face_data=[]
dataset_path='./data/'
file_name_person=input("enter the name:")
file_roll_person=input("enter the roll number:")
df = pd.read_csv('attendace.csv')
print(df)
data = {
    "Names" : [str(file_name_person)],
    "Roll Number" :[ str(file_roll_person)]
}
add_df = pd.DataFrame(data)
new_df  = df.append(add_df)
new_df.to_csv('attendace.csv',index=False)

file_name = str(file_name_person) + str(file_roll_person)

while True:
	ret,frame=cap.read()
	gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

	if ret==False:
		continue

	faces=face.detectMultiScale(frame,1.3,5)
	faces=sorted(faces,key=lambda f:f[2]*f[3])

	for (x,y,w,h) in faces[-1:]:
		cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)

		offset=10
		face_section=frame[y-offset:y+h+offset,x-offset:x+w+offset]
		face_section=cv2.resize(face_section,(100,100))

		skip+=1
		if skip%10==0:
			face_data.append(face_section)
			print(face_data)

	cv2.imshow("frame",frame)
	#cv2.imshow("face_section",face_section)

	key=cv2.waitKey(30) & 0xFF

	if key==ord('q'):
		break	

face_data=np.asarray(face_data)
face_data=face_data.reshape((face_data.shape[0],-1))
#print(face_data.shape)
new_df = pd.read_csv('attendace.csv')
columns_list = np.array(new_df.columns)
drop_col = []
for col in columns_list:
    if "Unnamed:" in  col:
        drop_col.append(col)
new_df.drop(drop_col,axis = 1,inplace=True)
new_df.fillna(0,inplace=True)
new_df.to_csv('attendace.csv',index=False)
new_df = pd.read_csv('attendace.csv')

print(new_df.head())

np.save(dataset_path+file_name+".npy",face_data)
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
import pandas as pd
import os
def dist(x1,x2):
    return np.sqrt(sum(((x1-x2)**2)))
new_df = pd.read_csv('attendace.csv')
current_students = [ ]
student_id = ' '
for i in range(new_df.shape[0]):
    student_id  = new_df['Names'].values[i] + str(new_df['Roll Number'].values[i])
    current_students.append(student_id)
print(current_students)
def knn(X,Y,k=5):
    val=[]
    m=X.shape[0]
    for i in range(m):
        ix=X[i,:-1]
        iy=X[i,-1]

        d=dist(Y,ix)
        val.append((d,iy))
    vals=sorted(val,key=lambda x:x[0])[:k]
    vals=np.array(vals)[:,-1]
    
    new_val=np.unique(vals,return_counts=True)
    #print(new_val)
    index=np.argmax(new_val[1])
    pred=new_val[0][index]
    return pred


cap=cv2.VideoCapture(0)

face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_alt2.xml")

skip=0
face_data=[]
dataset_path='./data/'
label=[]
class_id=0
names={}

for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):

        names[class_id]=fx[:-4]

        print("loaded "+fx)
        data_item=np.load(dataset_path+fx)
        face_data.append(data_item)


        #Create labels for class
        target=class_id*np.ones((data_item.shape[0],))
        class_id+=1
        label.append(target)


face_dataset=np.concatenate(face_data,axis=0)
labels_dataset=np.concatenate(label,axis=0).reshape((-1,1))

trainset=np.concatenate((face_dataset,labels_dataset),axis=1)

while True:
    ret,frame=cap.read()

    if ret==False:
        continue

    faces=face_cascade.detectMultiScale(frame,1.3,5)

    for face in faces:
        x,y,w,h=face

        offset=10

        face_section=frame[y-offset:y+h+offset,x-offset:x+offset+w]
        face_section=cv2.resize(face_section,(100,100))
        print(face_section)
        out=knn(trainset,face_section.flatten())
        pred=names[int(out)]
        if pred in current_students:
            print(pred)
            pred = pred + 'Access Allowed'
        else:
            pred = pred + 'Access Denied'
        cv2.putText(frame,pred,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,1),1,cv2.LINE_AA)

        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)

    cv2.imshow("frame",frame)
    key=cv2.waitKey(1) & 0xFF
    if key==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
!pip3 install wxpython


In [ ]:
import cv2
import numpy as np
import wx
from pynput.mouse import Button, Controller

from func import *

mouse = Controller()

app = wx.App(False)
(screenx, screeny) = wx.GetDisplaySize()
(capturex, capturey) = (700, 500)  # captures this size frame

cap = cv2.VideoCapture(0)


cap.set(3, capturex)
cap.set(4, capturey)

kernelOpen = np.ones((5, 5))  # if noise are present other than yellow area
kernelClose = np.ones((20, 20))  # if noise are present in yellow area

# HSV color range which should be detected
lb = np.array([20, 100, 100])
ub = np.array([120, 255, 255])

cd = 0

while True:
    ret, frame = cap.read()

    # use HSV of yellow to detect only yellow color
    imgSeg = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    # imgSegFlipped = cv2.resize(imgSegFlipped,(500,400))

    # masking and filtering all shades of yellow
    mask = cv2.inRange(imgSeg, lb, ub)
    # mask = cv2.resize(mask,(500,400))

    # apply morphology to avoid noise
    maskOpen = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernelOpen)
    # maskOpen = cv2.resize(maskOpen,(500,400))
    maskClose = cv2.morphologyEx(maskOpen, cv2.MORPH_CLOSE, kernelClose)
    # maskClose = cv2.resize(maskClose,(500,400))

    final = maskClose
    conts, h = cv2.findContours(maskClose, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    if (len(conts) != 0):  # draws the contours of that object which has the highest
        b = max(conts, key=cv2.contourArea)
        west = tuple(b[b[:, :, 0].argmin()][0])  # gives the co-ordinate of the left extreme of contour
        east = tuple(b[b[:, :, 0].argmax()][0])  # above for east i.e right
        north = tuple(b[b[:, :, 1].argmin()][0])
        south = tuple(b[b[:, :, 1].argmax()][0])
        centre_x = (west[0] + east[0]) / 2
        centre_y = (north[0] + south[0]) / 2

        cv2.drawContours(frame, b, -1, (0, 255, 0), 3)
        cv2.circle(frame, west, 6, (0, 0, 255), -1)
        cv2.circle(frame, east, 6, (0, 0, 255), -1)
        cv2.circle(frame, north, 6, (0, 0, 255), -1)
        cv2.circle(frame, south, 6, (0, 0, 255), -1)
        cv2.circle(frame, (int(centre_x), int(centre_y)), 6, (255, 0, 0), -1)  # plots centre of the area

        bint = int(cv2.contourArea(b))

        if (bint in range(8000, 18000)):  # hand is open
            mouse.release(Button.left)
            cv2.circle(frame, (int(centre_x), int(centre_y)), 6, (255, 0, 0), -1)  # plots centre of the area
            mouse.position = (screenx - (centre_x * screenx / capturex), screeny - (centre_y * screeny / capturey))

        elif (bint in range(2000, 7000)):  # hand is closed
            cv2.circle(frame, (int(centre_x), int(centre_y)), 10, (255, 255, 255), -1)  # plots centre of the area
            mouse.position = (screenx - (centre_x * screenx / capturex), screeny - (centre_y * screeny / capturey))
            mouse.press(Button.left)

    cv2.imshow('video', frame)
    if cv2.waitKey(1) & 0xFF == ord(' '):  # exiting
        break

cap.release()
cv2.destroyAllWindows()